<a href="https://colab.research.google.com/github/micuentadecasa/masakhane-mt/blob/Sango/sango2english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "sg"
target_language = "en" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# # This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
# os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [4]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/sg-en-baseline


In [5]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 3.2MB/s 


In [6]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q
# extract the corpus file
! gunzip JW300_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sg.xml.gz not found. The following files are available for downloading:

   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-sg.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  33 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/sg.zip

 299 MB Total size
./JW300_latest_xml_en-sg.xml.gz ... 100% of 3 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_sg.zip ... 100% of 33 MB


In [7]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df1= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df1.shape)
df1.head(8)

Loaded data and skipped 0/289194 lines since contained in test set.
(289195, 2)


,source_sentence,target_sentence
0,,The Bible’s Viewpoint
1,“ I yeke ti sese so pëpe ” : Tongana nyen ?,“ No Part of the World ” — What Does It Mean ?
2,"NA SIÈCLE osio ti ngoi ti e , azo saki mingi s...","IN THE fourth century C.E . , thousands of pro..."
3,Na salango tongaso ala ga a - anachorète .,"They became known as anchorites , from the Gre..."
4,"Tënë so alondo na tënë anakorêtès na Grec , so...",One historian describes them as holding themse...
5,Mbeni wasungo - mbaï atene so ala ke songo kue...,
6,"Na zingo tele ti ala yamba na tanga ti azo , a...",Anchorites thought that by withdrawing from hu...
7,A yeke tâ tënë so Bible awa aChrétien ti ‘ bat...,The Bible does admonish Christians to keep “ w...


In [8]:
# Downloading our corpus
! opus_read -d Tatoeba -s $src -t $tgt -wm moses -w tatoeba.$src tatoeba.$tgt -q

# extract the corpus file
! gunzip Tatoeba_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/Tatoeba/latest/xml/en-sg.xml.gz not found. The following files are available for downloading:

   4 KB https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-11-09/xml/en-sg.xml.gz
 109 MB https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-11-09/xml/en.zip
   8 KB https://object.pouta.csc.fi/OPUS-Tatoeba/v2020-11-09/xml/sg.zip

 109 MB Total size
./Tatoeba_latest_xml_en-sg.xml.gz ... 100% of 4 KB
./Tatoeba_latest_xml_en.zip ... 100% of 109 MB
./Tatoeba_latest_xml_sg.zip ... 100% of 8 KB


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'tatoeba.' + source_language
target_file = 'tatoeba.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df2= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df2.shape)
df2.head(3)

Loaded data and skipped 0/10 lines since contained in test set.
(11, 2)


,source_sentence,target_sentence
0,Ïrï tî mo nyen ?,What is your name ?
1,Balaô !,Hello !
2,Singîla !,Thank you !


In [10]:
# Downloading our corpus
! opus_read -d wikimedia -s $src -t $tgt -wm moses -w wikimedia.$src wikimedia.$tgt -q

# extract the corpus file
! gunzip wikimedia_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/wikimedia/latest/xml/en-sg.xml.gz not found. The following files are available for downloading:


   0 KB Total size
No alignment file "/proj/nlpl/data/OPUS/wikimedia/latest/xml/en-sg.xml.gz" or "./wikimedia_latest_xml_en-sg.xml.gz" found
gzip: wikimedia_latest_xml_en-sg.xml.gz: No such file or directory


In [11]:
# TMX file to dataframe
source_file = 'wikimedia.' + source_language
target_file = 'wikimedia.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df3= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df3.shape)
df3.head(3)

Loaded data and skipped 0/10 lines since contained in test set.
(0, 2)


,source_sentence,target_sentence


In [12]:
# Downloading our corpus
! opus_read -d Tanzil -s $src -t $tgt -wm moses -w tanzil.$src tanzil.$tgt -q

# extract the corpus file
! gunzip Tanzil_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/Tanzil/latest/xml/en-sg.xml.gz not found. The following files are available for downloading:


   0 KB Total size
No alignment file "/proj/nlpl/data/OPUS/Tanzil/latest/xml/en-sg.xml.gz" or "./Tanzil_latest_xml_en-sg.xml.gz" found
gzip: Tanzil_latest_xml_en-sg.xml.gz: No such file or directory


In [13]:
# TMX file to dataframe
source_file = 'tanzil.' + source_language
target_file = 'tanzil.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df4= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df4.shape)
df4.head(3)

Loaded data and skipped 0/10 lines since contained in test set.
(0, 2)


,source_sentence,target_sentence


In [14]:
# in our case for sango the df3 and df4 are empty
List = [df1, df2, df3, df4]  # List of your dataframes
df = pd.concat(List)

In [15]:
df.shape

(289206, 2)

In [16]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
#df_pp.drop_duplicates(subset='source_sentence', inplace=True)
#df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [17]:
df_pp.shape

(274449, 2)

In [19]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv
import numpy as np

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

#Divide df_pp into train and test sets

msk = np.random.rand(len(df_pp)) < 0.98
train = df_pp[msk]
test_df = df_pp[~msk]

# Further divide train into train and additional dev set which should be the last 1000
dev_df = train.tail(num_dev_patterns) # Herman: Error in original
stripped_df = train.drop(train.tail(num_dev_patterns).index)

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*
! head test.*

==> train.en <==
2 God Does Not Really Care ​ — Is It True ?
In the following article , we will expand on aspects of discipline within the family and the congregation .
Six branches were combined under the direction of the Mexico branch
Eber
We recognize maturity in a person when we see discernment , insight , wisdom , and so on .
Paul and Silas were able to give a fine witness to the jailer and his family .
Instead of thinking that we always know best , may we humbly listen to wise counselors who seek to help us by using the Scriptures .
• Why is it helpful to preach to people in their mother tongue ?
And because I am more available to help with the children , Ruth has been able to auxiliary pioneer from time to time .
The inspired account states : “ After that God saw everything he had made and , look !

==> train.sg <==
2 Nzapa ayeke bi bê ti lo biani na e pëpe : A yeke tâ tënë la ?
Na yâ ti article ti peko , e yeke sara tënë mingi na ndö ti fango lege na yâ ti sewa nga na congrégat

In [20]:
stripped_df.to_csv('train.csv')
!cp train.csv drive/My\ Drive/masakhane/

In [21]:
test_df.to_csv('test.csv')
!cp test.csv drive/My\ Drive/masakhane/

In [22]:

dev_df.to_csv('dev.csv')
!cp dev.csv drive/My\ Drive/masakhane/

In [ ]:

# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2724 (delta 4), reused 4 (delta 1), pack-reused 2711
Receiving objects: 100% (2724/2724), 2.80 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (1865/1865), done.
Processing /content/joeynmt
     |███████████████████████████████▉| 745.8MB 1.4MB/s eta 0:00:03

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt


# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt


In [ ]:
# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

In [ ]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

In [ ]:

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Hausa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
#! tail -n 10 joeynmt/data/$src$tgt/vocab.txt
! tail -n 10 "$gdrive_path/vocab.txt" #lmm

In [ ]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint


In [ ]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)


# very important !!!!!!!!!!!!!!!!!!!!!!!!!
# uncomment line load_model if you want to use a checkpoint ..

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, last_checkpoint=last_checkpoint, model_temp_dir=model_temp_dir, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [ ]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
# # Copy the created models from the notebook storage to google drive for persistant storage 
# !cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

In [ ]:
# python3 plot_validation.py "$gdrive_path/models/${src}${tgt}_transformer/" --plot_values bleu PPL --output_path my_plot.pdf